In [21]:
import pandas as pd
import os
from datetime import datetime
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import json
import pickle
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score, roc_auc_score, classification_report, roc_auc_score,precision_score,recall_score

# import warnings
# warnings.filterwarnings('ignore')

In [22]:
data_raw_source = 'Датасеты'
data_postprocessed_source = 'postprocess_dataset'
models_source = "models"


houses_raw_source = f'{data_raw_source}/1_Многоквартирные дома с технико-экономическими характеристиками.xlsx'
houses_postprocess_source = f'{data_postprocessed_source}/1.csv'
appeals_postprocess_source = f'{data_postprocessed_source}/2.csv'
works_postprocess_source = f'{data_postprocessed_source}/3.csv'

houses_test_source = f'{data_postprocessed_source}/houses_test.csv'

geo_source = f'{data_postprocessed_source}/geo.csv'

imply_appeals_work_without_time_weight_source = f'{data_postprocessed_source}/imply_appeals_work_without_time_weight.pkl'
imply_appeals_work_with_time_weight_source = f'{data_postprocessed_source}/imply_appeals_work_with_time_weight.pkl'

works_time_cols = ['PLAN_DATE_START', 'PLAN_DATE_END', 'FACT_DATE_START', 'FACT_DATE_END']
datetime_str_start = '2022-01-01 00:00:00.000000'
date_format = '%Y-%m-%d %H:%M:%S.%f'
date_format3 = '%d.%m.%Y'
datetime_start = datetime.strptime(datetime_str_start, date_format)

houses_columns = ['ID', 'NAME', 'PARENT_ID', 'LOGIN', 'Назначение', 'Форма собственности',
       'Год постройки', 'Год реконструкции', 'Серия проекта',
       'Количество этажей', 'Количество подъездов', 'Количество квартир',
       'Общая площадь', 'Общая площадь жилых помещений',
       'Общая площадь нежилых помещений', 'Строительный объем',
       'Износ объекта (по БТИ)', 'Класс энергоэффективности', 'Материал стен',
       'Признак аварийности здания', 'Количество пассажирских лифтов',
       'Количество грузопассажирских лифтов', 'Очередность уборки кровли',
       'Материал кровли', 'UNOM', 'Вид социального объекта',
       'Тип жилищного фонда', 'Статус МКД', 'Статус управления МКД',
       'Количество грузовых лифтов', 'Причина Изменения Статуса МКД',
       'Категория МКД']

appeals_columns = ['Наименование', 'Источник', 'Дата создания во внешней системе',
       'Дата закрытия', 'Округ', 'Адрес', 'unom',
       'Дата и время завершения события во']

works_columns = ['global_id', 'PERIOD', 'WORK_NAME', 'NUM_ENTRANCE', 'ElevatorNumber',
       'PLAN_DATE_START', 'PLAN_DATE_END', 'FACT_DATE_START', 'FACT_DATE_END',
       'AdmArea', 'District', 'Address', 'UNOM']
WORK_WEIGHTS = {
    "ремонт крыши": 10,
    "ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)": 9,
    "ремонт внутридомовых инженерных систем теплоснабжения (стояки)": 9,
    "ремонт внутридомовых инженерных систем газоснабжения": 9,
    "ремонт внутридомовых инженерных систем электроснабжения": 8,
    "замена лифтового оборудования": 8,
    "ремонт внутридомовых инженерных систем водоотведения (канализации) (стояки)": 8,
    "ремонт внутридомовых инженерных систем холодного водоснабжения (стояки)": 8,
    "ремонт пожарного водопровода": 7,
    "ремонт фасадов": 7,
    "ремонт внутридомовых инженерных систем водоотведения (канализации) (выпуски и сборные трубопроводы)": 7,
    "ремонт внутридомовых инженерных систем горячего водоснабжения (разводящие магистрали)": 6,
    "ремонт внутридомовых инженерных систем холодного водоснабжения (разводящие магистрали)": 6,
    "ремонт внутридомовых инженерных систем теплоснабжения (разводящие магистрали)": 6,
    "ремонт подъездов, направленный на восстановление их надлежащего состояния и проводимый при выполнении иных работ по капитальному ремонту общего имущества в многоквартирном доме": 5,
    "ремонт внутреннего водостока": 5,
    "ремонт подвальных помещений, относящихся к общему имуществу в многоквартирном доме": 4,
    "ремонт мусоропровода": 3,
    "замена оконных блоков, расположенных в помещениях общего пользования": 2
}


In [23]:
all_appeals = ['Отсутствие связи', 'Запах газа в подъезде и на улице', 'Запах газа от ШБДГ', 'Загрязнение/замусоренность подъезда', 'Повреждение козырька подъезда', 'Хлопок газа', 'Нарушение подачи воды, слабое давление', 'Колодцы, провал колодца', 'Наличие грибка/плесени', 'Запах газа в квартире (помещении); Запах газа от газового оборудования', 'Открыт колодец МГ', 'Нет газа', 'стучит крышка ковера', 'Колодцы, хлопает крышка', 'Нет  газа', 'Отсутствие освещения в лифте', 'Запах газа в коллекторе', 'Нарушение в работе газового оборудования; Слабое горение', 'Отрицательные интегральные значения', 'Гремит  крышка ковера', 'Повреждение межпанельных швов', 'Повреждение внутренней двери', 'Аномальное значение разницы объемов', 'Качество воды - мутная вода', 'Отсутствуют актуальные мгновенные значения', 'Повреждение/поломка светильника', 'Запах газа из закрытой квартиры', 'Запах в подъезде', 'Повреждение элементов входной двери', 'Слабое горение', 'Погас Вечный огонь', 'T1 > max', 'Угроза взрыва', 'Аномальное значение температуры в обратном трубопроводе', 'Неочищенная кровля', 'Отклонение ГВС ниже нормы днем (мониторинг)', 'Колодцы, люк на боку', 'Запах газа в кухне; Запах газа в квартире (помещении)', 'Повреждение системы электропроводки/щитового оборудования', 'Запах газа от настенного газопровода', 'Гремят крышки коверов', 'Провал грунта', 'Температура в помещении общего пользования ниже нормативной', 'Загрязнение воды', 'Неудовлетворительное санитарное содержание мусоропровода', 'Провалы, прочее', 'Проверить настенный газопровод', 'Датчик вибрации', 'Аномальное значение отпущенной тепловой энергии', 'Запах  газа  на  улице и в подъезде', 'Повреждение кровли', 'Запах газа по вентиляции', 'Низкий уровень сигнала GSM', 'Прорыв теплосети (водопровода); Задымление', 'Открыт колодец на газоне', 'Нет связи с ПУ', 'Утечка воды, течет спец. вывод', 'Запах газа у АГВ', 'Недогрев ГВС', 'Аварийная протечка в подъезде', 'Подтопление, прочее', 'Запах газа в туалете', 'Аномальное значение объема в обратном трубопроводе', 'Нет газа в кваптире', 'Поломка пандуса', 'Открыта крышка колодца (трещит крышка ковера)', 'Пожар; Обрушение', 'Нет газа в квартире', 'Отклонение ГВС ниже нормы ночью (мониторинг)', 'Отсутствует циркуляция ГВС', 'Нарушение на настенном газопроводе', 'Загрязнение почтовых ящиков', 'Засор на городской сети', 'Нет связи с УСПД', 'Неисправность пожарной сигнализации', 'Загрязнение входной двери', 'Поломка почтовых ящиков', 'Утечка воды из-под асфальта, поступает на проезжую часть', 'Открата крышка ковера', 'Повреждение элементов фасада', 'Вибрирует газовая труба', 'Нарушение на настенном газопроводе; Запах газа в квартире (помещении)', 'Повреждение отделочных покрытий пола/стены/ступеней/перил/других элементов', 'Засор канализации', 'Изменение канала связи', 'Нет  газа в квартире', 'Запах газа на улице и в квартире', 'Пожар', 'Разбито/сломано/повреждено окно в местах общего пользования', 'Провал асфальтобетонного покрытия', 'Пожар; Запах газа в квартире (помещении)', 'Провис настенный газопровод', 'Открыт колодец; Провал', 'Протечка в подъезде', 'Колодцы на проезжей части, провал колодца', 'Запах газа в квартире (помещении); Запах газа в доме', 'Засор мусоропровода', 'Наличие надписей/объявлений', 'Восстановление трубопровода, пневмопробойник', 'Запах газа в холле и квартире', 'Обрушение', 'Колодцы, люк завышен', 'Отсутствует крышка ковера', 'Запах газа от газового оборудования', 'Лифт требует ремонта', 'Колодцы, разрушен асфальт вокруг колодца', 'Повреждение асфальтобетонного покрытия', 'Загрязнение отделочных покрытий', 'Подтопление, течь задвижки на водомерном узле', 'Повреждение инженерных сетей', 'Засоры, запах', 'Качество воды - окрашенная вода', 'Загазованность крана № Г2 40162 2,6%', 'Провал', 'Запах газа в квартире с улицы', 'Проверить квартиру после пожара', 'Запах газа от колодца; Открыт колодец', 'Запах газа в подъезде и в квартире', 'Качество воды - вода с запахом', 'Загрязнение лифта', 'Изменение конфигурации УСПД', 'Аварийная протечка с кровли', 'Гремит крышка ковера', 'Сильная течь в системе отопления', 'Запах газа в ванной', 'Загрязнение/замусоренность территории', 'Качество воды - ржавая вода', 'Блокировка входной двери', 'Температура в квартире ниже нормативной', 'P1 <= 0', 'Запах газа в кухне; Запах газа в доме', 'Просадка асфальта вокруг колодца', 'Запах газа в подъезде и в квартире.', 'Подтопление канала теплосети', 'Гул (шум) на объекте АО МОСГАЗ; Запах газа на улице', 'Отсутствие отопления в доме', 'Открыт шкаф УСПД', 'Колодцы на проезжей части, люк завышен', 'Течь в системе отопления', 'Критичное отклонение температуры ГВС ниже нормы ночью (мониторинг)', 'Описание отсутствует', 'Колодцы на проезжей части, открыт колодец', 'Прорыв теплосети (водопровода)', 'Пожар; Запах газа в кухне', 'Провал на газоне', 'Запах газа в подъезде', 'Колодцы на проезжей части, разрушен асфальт вокруг колодца', 'Колодцы, люк занижен', 'Провал на месте старой раскопки', 'Подтопление телефонной сети', 'Колодцы, прочее', 'Повреждение плиточного/брусчатого покрытия', 'Неработоспособность подъемной платформы для инвалидов', 'Запах газа в квартире (помещении)', 'Нарушение подачи воды, прочее', 'Отсутствие ГВС в доме', 'Неудовлетворительное техническое содержание мусоропровода', 'Температура ГВС ниже нормы', 'Провал колодца', 'Загрязнение/замусоренность подвала/полуподвала', 'Запах газа в холле', 'Подтопление строений', 'Загрязнение/замусоренность козырька', 'Нарушение в работе газового оборудования', 'Аномальное значение массы в подающем трубопроводе', 'Не  работает  АГВ', 'Задымление в подъезде', 'Колодцы, люк расколот', 'Колодцы, нет люка и крышки', 'Нарушение подачи воды, не работает водопроводная колонка', 'Утечка воды из колодца, прочее', 'Хлопает крышка ковера', 'Колодцы, люк сдвинут', 'Аварийное повреждение лестницы', 'Подтопление сооружений метрополитена', 'Нарушение подачи воды, нет воды на верхних этажах', 'Повреждение подъемной платформы для инвалидов', 'Занижен ковер', 'Нарушение в работе газового оборудования; Запах газа в кухне', 'Утечка воды из колодца, поступает на проезжую часть', 'Провал плиточного/брусчатого покрытия', 'Загрязнение окна в местах общего пользования', 'Запах газа в кухне от стояка', 'Утечка воды из колодца, поступает в канализацию/водосток', 'Подтопление приямка ЦТП', 'Подтопление, поступает вода в камеру теплосети', 'P2 <= 0', 'Восстановление трубопровода, просадка грунта на трассе трубопровода', 'Запах газа от колонки', 'Пожар; Задымление', 'Нарушение на настенном газопроводе; Гул (шум, вибрация) от газопровода', 'Подтопление, повреждение водопровода в городском коллекторе', 'Подтопление, повреждение водопровода во внутриквартальном коллекторе', 'Утечка воды из-под асфальта, прочее', 'Недокомплект пожарного шкафа', 'Гул (шум, вибрация) от газопровода', 'Проверить работу ЭХЗ', 'Взрыв; Пожар; Задымление', 'Нет газа по стояку', 'Неисправность запирающего устройства', 'Повреждение пола/стены/ступеней/перил/других элементов', 'Засоры, засор на дворовой сети', 'Отсутствие освещения в местах общего пользования', 'Прорыв теплосети (водопровода); Открыт колодец', 'Запах газа на улице, в подъезде', 'Отсутствует крышка колодца', 'Протечка с кровли', 'Аномальное значение разницы температур', 'Загрязнение внутренней двери', 'Запах газа в кухне', 'Колодцы на проезжей части, хлопает крышка', 'Открыт колодец на проезжей части', 'Течет вода из колонки.', 'Подтопление, вода поступает в котлован строителей', 'Запах гари в квартире/подъезде', 'Аварийная протечка труб в подъезде', 'T1 < min', 'Качество воды, прочее', 'Утечка воды из земли, поступает на проезжую часть', 'Запах газа из подвала', 'Несовпадение серийного номера ПУ на УСПД', 'Запах газа в квартире', 'Запах газа на улице', 'Утечка воды, прочее', 'Качество воды - вода с осадком', 'Взрыв', 'Запах газа в подъезде и квартире', 'Запах газа в помещении', 'Нарушение подачи воды, нет воды в здании', 'Открыт колодец', 'Нарушение на настенном газопроводе; Запах газа в кухне', 'Поломка лифта', 'Взрыв; Пожар', 'Задымление', 'Поломка освещения перед подъездом', 'Наличие крыс/мышей/насекомых в местах общего пользования', 'Запах газа в доме', 'Угроза обрушения', 'Завышено давление газа', 'Запах газа с улицы в квартиру', 'Аномальное значение температуры в подающем трубопроводе', 'Застревание в лифте', 'Расхождение времени ПУ', 'Протечка труб в подъезде', 'Пожар; Запах газа в доме', 'Открыта крышка колодца', 'Подтопление, течь трубопровода на водомерном узле', 'Аномальное значение объема в подающем трубопроводе', 'Утечка воды из земли, прочее', 'Протечка с балкона/козырька', 'Засоры, прочее', 'Запах газа от стояка', 'Критичное отклонение температуры ГВС ниже нормы днем (мониторинг)', 'Провал грунта вокруг ковера', 'Нет питания УСПД', 'Аномальное значение времени наработки', 'Провал на проезжей части', 'Отсутствуют актуальные суточные значения', 'Отсутствие ХВС в доме', 'Иные нарушения', 'Запах газа на улице; Запах газа в доме', 'Гул (шум) на объекте АО МОСГАЗ', 'Запах газа от газового оборудования; Запах газа в доме', 'Запах газа на улице и в подъезде', 'Запах газа в кухне; Запах газа от газового оборудования']
all_works = ['ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)', 'замена лифтового оборудования', 'ремонт пожарного водопровода', 'ремонт крыши', 'ремонт мусоропровода', 'ремонт внутридомовых инженерных систем газоснабжения', 'ремонт внутридомовых инженерных систем холодного водоснабжения (разводящие магистрали)', 'ремонт подвальных помещений, относящихся к общему имуществу в многоквартирном доме', 'замена оконных блоков, расположенных в помещениях общего пользования', 'ремонт внутридомовых инженерных систем водоотведения (канализации) (выпуски и сборные трубопроводы)', 'ремонт внутридомовых инженерных систем электроснабжения', 'ремонт фасадов', 'ремонт внутридомовых инженерных систем теплоснабжения (стояки)', 'ремонт внутридомовых инженерных систем холодного водоснабжения (стояки)', 'ремонт внутридомовых инженерных систем водоотведения (канализации) (стояки)', 'ремонт внутреннего водостока', 'ремонт внутридомовых инженерных систем горячего водоснабжения (разводящие магистрали)', 'ремонт внутридомовых инженерных систем теплоснабжения (разводящие магистрали)', 'ремонт подъездов, направленный на восстановление их надлежащего состояния и проводимый при выполнении иных работ по капитальному ремонту общего имущества в многоквартирном доме']

In [24]:
nessesery_cols = ['unom', 'NAME', 'Тип жилищного фонда']
base_features = ['Год постройки', 'Серия проекта', 'Количество этажей', 'Количество квартир', 'Количество подъездов',
                'Общая площадь', 'Общая площадь жилых помещений', 'Общая площадь нежилых помещений', 'Материал стен',
                 'Количество пассажирских лифтов', 'Количество грузопассажирских лифтов', 'Очередность уборки кровли',
                 'Материал кровли', 'Статус управления МКД', 'lng', 'lat']

base_cat_features = ['Серия проекта', 'Материал стен', 'Очередность уборки кровли', 'Материал кровли', 'Статус управления МКД']
time_features = ["x_length_time_period", "time_dist_period", "y_length_time_period"]

In [25]:
# дроп первого техничесного набора названий стобцов 
def read_sub_df(df):
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    return df


def preprocess_houses_dataset(houses_source, houses_raw_source):
    try:
        data = pd.read_excel(houses_source, sheet_name=None)
    except:
        data = pd.read_csv(houses_source)
    if isinstance(data, dict):
        key = list(data)[0]
        data = data[key]
    if list(data.columns) == houses_columns:
        return data
    df = pd.read_excel(houses_raw_source, sheet_name=None)
    col_names = [i for i in df if i != "Sheet1"]
    col_dict = df['Sheet1'].iloc[0].to_dict()
    for i in col_dict:
        if pd.isna(col_dict[i]):
            col_dict[i] = i
    data = data.drop(data.index[0])
    data.columns = [col_dict[i] for i in data.columns]

    for name in col_names:
        df[name] = read_sub_df(df[name])
        d = df[name].astype(str).set_index('ID').to_dict()['NAME']
        data[col_dict[name]] = data[col_dict[name]].apply(lambda x: d.get(x, None))
    return data

def preprocess_appeals_dataset(appeals_source):
    try:
        data = pd.read_excel(appeals_source, sheet_name=None)
    except:
        data = pd.read_csv(appeals_source)
        return data
    if not isinstance(data, dict):
        return data
    data_arr = []
    for i in data:
        if list(data[i].columns) == appeals_columns:
            data_arr.append(i)
    return pd.concat([data[i] for i in data_arr]).drop_duplicates()

def preprocess_works_dataset(works_source):
    try:
        return pd.read_excel(works_source)
    except:
        return pd.read_csv(works_source)
        
def get_datasets(houses_source, appeals_source, works_source):
    houses_df = preprocess_houses_dataset(houses_source, houses_raw_source)
    appeals_df = preprocess_appeals_dataset(appeals_source)
    works_df = preprocess_works_dataset(works_source)
    return houses_df, appeals_df, works_df

In [26]:
def preprocess(houses_df, appeals_df, works_df, geo, only_mkd=True):
    if only_mkd:
        houses_df_new = houses_df[(houses_df['Тип жилищного фонда'] == 'МКД') & (houses_df['Статус МКД'] == 'в эксплуатации')]
    else:
        houses_df_new = houses_df.copy()
    houses_df_new = houses_df_new.rename(columns={"UNOM": "unom"})
    
    
    houses_df_new['unom'] = houses_df_new['unom'].astype(int)
    geo['unom'] = geo['unom'].astype(int)
    houses_df_new = houses_df_new.set_index('unom').join(geo.set_index('unom')).reset_index()
    houses_df_new = houses_df_new[nessesery_cols + base_features]
    
    appeals_df_new = appeals_df.copy()
    appeals_df_new['unom'] = appeals_df_new['unom'].astype(int)
    appeals_df_new = appeals_df_new[appeals_df_new['unom'].isin(houses_df_new['unom'])].reset_index(drop=True)
    
    appeals_df_new['Дата создания во внешней системе'] = appeals_df_new['Дата создания во внешней системе'].apply(lambda x: (datetime.strptime(x, date_format) - datetime_start).total_seconds() / (3600 * 24))
    appeals_df_new['Дата закрытия'] = appeals_df_new['Дата закрытия'].apply(lambda x: (datetime.strptime(x, date_format) - datetime_start).total_seconds() / (3600 * 24) if not pd.isna(x) else None)
    appeals_df_new['Дата и время завершения события во'] = appeals_df_new['Дата и время завершения события во'].apply(lambda x: (datetime.strptime(x, date_format) - datetime_start).total_seconds() / (3600 * 24) if not pd.isna(x) else None)
    
    grouped = appeals_df_new.groupby('unom').apply(lambda x: x[['Наименование', 'Дата создания во внешней системе', 'Дата закрытия', 'Дата и время завершения события во']].to_dict('records'))
    appeals_df_new = pd.DataFrame({'unom': grouped.index, 'appeals_list': grouped.values})
    appeals_df_new['appeals_list'] = appeals_df_new['appeals_list'].apply(lambda x: sorted(x, key = lambda y: y['Дата создания во внешней системе']))
    houses_df_new = houses_df_new.set_index('unom').join(appeals_df_new.set_index('unom')).reset_index()
    
    works_df_new = works_df.rename(columns={"UNOM": "unom"})
    works_df_new['unom'] = works_df_new['unom'].astype(int)
    for i in works_time_cols:
        works_df_new[i] = works_df_new[i].apply(lambda x: (datetime.strptime(x, date_format3) - datetime_start).total_seconds() / (3600 * 24) if not pd.isna(x) else None)
    grouped = works_df_new.groupby('unom').apply(lambda x: x[works_time_cols + ['WORK_NAME']].to_dict('records'))
    works_df_new = pd.DataFrame({'unom': grouped.index, 'works_list': grouped.values})
    works_df_new['works_list'] = works_df_new['works_list'].apply(lambda x: sorted(x, key = lambda y: y['PLAN_DATE_START']))
    houses_df_new = houses_df_new.set_index('unom').join(works_df_new.set_index('unom')).reset_index()
    return houses_df_new

def split_data(data, imply_dict_size=1, train_size=1, test_size=1):
    sum_sizes = imply_dict_size + train_size + test_size
    imply_dict_size, train_size, test_size = imply_dict_size / sum_sizes, train_size / sum_sizes, test_size / sum_sizes
    imply_dict_data, temp_data = train_test_split(data, test_size=(train_size + test_size), random_state=42)
    train_data, test_data = train_test_split(temp_data, test_size=test_size / (train_size + test_size), random_state=42)
    return imply_dict_data.reset_index(drop=True), train_data.reset_index(drop=True), test_data.reset_index(drop=True)
    

def check_bounds(number, left_bound=None, right_bound=None):
    if pd.isna(left_bound):
        left_bound = number - 1
    if pd.isna(right_bound):
        right_bound = number + 1
    return left_bound < number < right_bound

def filter_bounds(df, left_bound=None, right_bound=None, type_l="appeals_list"):
    df_new = df.copy()
    if type_l == "appeals_list":
        df_new[type_l] = df_new[type_l].apply(lambda x: [y for y in x if check_bounds(y["Дата создания во внешней системе"],
                                                                                      left_bound=left_bound,
                                                                                      right_bound=right_bound)] if isinstance(x, list) else None)
    elif type_l == "works_list":
        df_new[type_l] = df_new[type_l].apply(lambda x: [y for y in x if check_bounds(y["PLAN_DATE_START"],
                                                                                      left_bound=left_bound,
                                                                                      right_bound=right_bound)] if isinstance(x, list) else None)
    return df_new

def get_imply_appeals_work(df, start_x_date=-1, end_x_date=9999, start_y_date=-1, end_y_date=9999, weight_func=None):
    if pd.isna(weight_func):
        weight_func = lambda x, y: 1
    norm_dict = {x: 0 for x in all_appeals}
    res_dict = {}
    for x in all_appeals:
        for y in all_works + ["without work"]:
            res_dict[(x, y)] = 0
    df_appeals = filter_bounds(df, start_x_date, end_x_date, type_l="appeals_list")
    df_works = filter_bounds(df, start_y_date, end_y_date, type_l="works_list")
    for i in range(len(df)):
        if not isinstance(df_appeals["appeals_list"][i], list) or not df_appeals["appeals_list"][i]:
            continue
        for appeal in df_appeals["appeals_list"][i]:
            if pd.isna(appeal["Наименование"]):
                continue
            norm_dict[appeal["Наименование"]] += 1
            if not isinstance(df_works["works_list"][i], list):
                res_dict[(appeal["Наименование"], "without work")] += 1
                continue
            works_list_filtered = list(filter(lambda x: appeal["Дата создания во внешней системе"] < x["PLAN_DATE_START"],
                                              df_works["works_list"][i]))
            if not works_list_filtered:
                res_dict[(appeal["Наименование"], "without work")] += 1
            else:
                loc_dict = {}
                loc_norm = 0
                for work in works_list_filtered:
                    if work["WORK_NAME"] not in loc_dict:
                        loc_dict[work["WORK_NAME"]] = 0
                    dist = weight_func(appeal["Дата создания во внешней системе"], work["PLAN_DATE_START"])
                    loc_dict[work["WORK_NAME"]] += dist
                    loc_norm += dist
                for key in loc_dict:
                    loc_dict[key] /= loc_norm
                    res_dict[(appeal["Наименование"], key)] += loc_dict[key]
    for x, y in res_dict:
        if not norm_dict[x]:
            continue
        res_dict[(x, y)] /= norm_dict[x]
    return res_dict

def create_imply_dict_data(imply_dict):
    df = pd.DataFrame(index=all_appeals, columns=all_works + ["without work"])
    for (appeal, work), value in imply_dict.items():
        df.loc[appeal, work] = value
    return df

def check_work_in(work, work_list):
    return any([work == x["WORK_NAME"] for x in work_list])

def get_last_work_time(work, work_list, end_x_date):
    if work_list is None or len(work_list) == 0:
        return 9999
    else:
        start_times = [d['PLAN_DATE_START'] for d in work_list if d['WORK_NAME'] == work and not pd.isna(d['PLAN_DATE_START'])]
        return end_x_date - max(start_times) if start_times else 9999
    
def get_all_appeals_after_date(appeals_list, date):
    res_appeals = []
    for i in range(len(appeals_list) - 1, -1, -1):
        if pd.isna(appeals_list[i]["Наименование"]):
            continue
        elif appeals_list[i]["Дата создания во внешней системе"] > date:
            res_appeals.append(appeals_list[i]["Наименование"])
        else:
            break
    return res_appeals


def generate_x(df, start_x_date, end_x_date):
    x = filter_bounds(df, start_x_date, end_x_date, type_l="appeals_list")
    x = filter_bounds(x, start_x_date, end_x_date, type_l="works_list")
    with open(imply_appeals_work_without_time_weight_source, 'rb') as infile:
        imply_appeals_work_without_time_weight_dict = pickle.load(infile)
    
    with open(imply_appeals_work_with_time_weight_source, 'rb') as infile:
        imply_appeals_work_with_time_weight_dict = pickle.load(infile)
    
    
    final_features = ["appeals_count", "works_count"]
    x["appeals_count"] = x["appeals_list"].apply(lambda x: len(x) if isinstance(x, list) else 0)
    x["works_count"] = x["works_list"].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
    for work in all_works:
        x[work + "_last_time_lag"] = x["works_list"].apply(lambda x: get_last_work_time(work, x, end_x_date))
        final_features.append(work + "_last_time_lag")
        x[work + "_without_time_weight_sum"] = x["appeals_list"].apply(lambda x: sum([imply_appeals_work_without_time_weight_dict[(y["Наименование"], work)] for y in x if not pd.isna(y["Наименование"])]) / (end_x_date - start_x_date + 1) if isinstance(x, list) else 0)
        final_features.append(work + "_without_time_weight_sum")
        x[work + "_without_time_weight_after_last_work_sum"] = x.apply(lambda x: sum([imply_appeals_work_without_time_weight_dict[(y, work)] for y in get_all_appeals_after_date(x["appeals_list"], end_x_date - x[work + "_last_time_lag"])] + [0]) / (end_x_date - start_x_date + 1) if isinstance(x["appeals_list"], list) else 0, axis=1)
        final_features.append(work + "_without_time_weight_after_last_work_sum")
        x[work + "_without_time_weight_after_last_work_max"] = x.apply(lambda x: max([imply_appeals_work_without_time_weight_dict[(y, work)] for y in get_all_appeals_after_date(x["appeals_list"], end_x_date - x[work + "_last_time_lag"])] + [0]) if isinstance(x["appeals_list"], list) else 0, axis=1)
        final_features.append(work + "_without_time_weight_after_last_work_max")
        
        x[work + "_with_time_weight_sum"] = x["appeals_list"].apply(lambda x: sum([imply_appeals_work_with_time_weight_dict[(y["Наименование"], work)] for y in x if not pd.isna(y["Наименование"])]) / (end_x_date - start_x_date + 1) if isinstance(x, list) else 0)
        final_features.append(work + "_with_time_weight_sum")
        x[work + "_with_time_weight_after_last_work_sum"] = x.apply(lambda x: sum([imply_appeals_work_with_time_weight_dict[(y, work)] for y in get_all_appeals_after_date(x["appeals_list"], end_x_date - x[work + "_last_time_lag"])] + [0]) / (end_x_date - start_x_date + 1) if isinstance(x["appeals_list"], list) else 0, axis=1)
        final_features.append(work + "_with_time_weight_after_last_work_sum")
        x[work + "_with_time_weight_after_last_work_max"] = x.apply(lambda x: max([imply_appeals_work_with_time_weight_dict[(y, work)] for y in get_all_appeals_after_date(x["appeals_list"], end_x_date - x[work + "_last_time_lag"])] + [0]) if isinstance(x["appeals_list"], list) else 0, axis=1)
        final_features.append(work + "_with_time_weight_after_last_work_max")
    return x, final_features

def generate_y(df, start_y_date, end_y_date):
    dff = filter_bounds(df, start_y_date, end_y_date, type_l="works_list")
    
    y = pd.DataFrame()
    for work in all_works:
        y[work] = dff["works_list"].apply(lambda x: check_work_in(work, x) if isinstance(x, list) else 0)
    return y

def greedy_max_distance(vectors, n_iter):
    selected_vectors = [vectors[0]] 
    for _ in tqdm(range(1, n_iter)): 
        min_distances = [min(np.linalg.norm(np.array(v) - np.array(selected_vector)) for selected_vector in selected_vectors) for v in vectors]
        selected_vectors.append(vectors[np.argmax(min_distances)])
    return selected_vectors

def generate_x_y(df, lower_bound_x_period_date=120, upper_bound_x_period_date=365,
                 lower_bound_y_period_date=1, upper_bound_y_period_date=365, y_period_positive_target_lower_bound=200, x_positive_lower_bound=100, n_iter_diff=50, n_iter_positive_target=50,
                 mult=100):
    x_out = []
    y_out = []
    
    vectors = []
    for _ in range(n_iter_diff * mult):
        start_x_date, end_x_date, start_y_date, end_y_date = -1, -1, -1, -1
        while not (lower_bound_x_period_date < end_x_date - start_x_date + 1 < upper_bound_x_period_date) or not (lower_bound_y_period_date < end_y_date - start_y_date + 1 < upper_bound_y_period_date):
            start_x_date, end_x_date, start_y_date, end_y_date = sorted(np.random.uniform(0, 365, size=4))
        vectors.append([start_x_date, end_x_date, start_y_date, end_y_date])
    selected_vectors = greedy_max_distance(vectors, n_iter_diff)
    
    vectors = []
    for _ in range(n_iter_positive_target * mult):
        start_x_date, end_x_date, start_y_date, end_y_date = -1, -1, -1, -1
        while not (x_positive_lower_bound < end_x_date - start_x_date + 1 < upper_bound_x_period_date) or not (y_period_positive_target_lower_bound < end_y_date - start_y_date + 1 < upper_bound_y_period_date):
            start_x_date, end_x_date, start_y_date, end_y_date = sorted(np.random.uniform(0, 365, size=4))
        vectors.append([start_x_date, end_x_date, start_y_date, end_y_date])
        
    selected_vectors += greedy_max_distance(vectors, n_iter_positive_target)
    
    for i in tqdm(range(n_iter_diff + n_iter_positive_target)):
        start_x_date, end_x_date, start_y_date, end_y_date = selected_vectors[i]
        x, final_features = generate_x(df, start_x_date, end_x_date)
        x["x_length_time_period"] = end_x_date - start_x_date
        x["time_dist_period"] = start_y_date - end_x_date
        x["y_length_time_period"] = end_y_date - start_y_date
        y = generate_y(df, start_y_date, end_y_date)
        x_out.append(x)
        y_out.append(y)
    return pd.concat(x_out).reset_index(drop=True), pd.concat(y_out).reset_index(drop=True), final_features

def cv_predictions(model, X, y, groups, n_splits=3):
    cv = GroupKFold(n_splits=n_splits)

    predictions = []
    test_labels = []
    for train_index, test_index in cv.split(X, y, groups):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        if not y_train.sum().astype(int) or not y_test.sum().astype(int):
            return None, None

        model.fit(X_train, y_train, cat_features=base_cat_features)
        y_pred_proba = model.predict_proba(X_test)
        predictions.extend(y_pred_proba)
        test_labels.extend(y_test)

    return np.array(predictions), np.array(test_labels)

def get_object_weight(x, p=False):
    if not p:
        return sum([WORK_WEIGHTS[y] for y in x["target"]] + [0])
    else:
        return sum([WORK_WEIGHTS[x["target"][i]] * x["p"][i] for i in range(len(x["target"]))] + [0])

def get_final_x_by_input(houses_postprocess_source, appeals_postprocess_source, works_postprocess_source, 
                         outname, resultsname, add_name, boundsname, configname):
    if not os.path.isfile(outname):
        geo = pd.read_csv(geo_source)
        houses_df, appeals_df, works_df = get_datasets(houses_postprocess_source, appeals_postprocess_source, works_postprocess_source)
        houses_df_new = preprocess(houses_df, appeals_df, works_df, geo)
        time_max = houses_df_new["appeals_list"].apply(lambda x: max([y['Дата создания во внешней системе'] for y in x]) if isinstance(x, list) else 0).max()
        with open(configname, "r") as file:
            config = json.load(file)
        start = (datetime.strptime(eval(config["dates"])[0], "%Y-%m-%dT%H:%M:%S.%fZ") - datetime_start).total_seconds() / (3600 * 24)
        end = (datetime.strptime(eval(config["dates"])[1], "%Y-%m-%dT%H:%M:%S.%fZ") - datetime_start).total_seconds() / (3600 * 24)
        x, final_features = generate_x(houses_df_new, 0, time_max)
        x["x_length_time_period"] = time_max - 0
        x["time_dist_period"] = start - time_max
        x["y_length_time_period"] = end - start
        x.to_csv(outname, index=False)
    if not os.path.isfile(resultsname):
        x = pd.read_csv(outname)
        res_data = pd.DataFrame()
        res_data["unom"] = x["unom"]
        res_data["description"] = x["NAME"]
        res_data["object_type"] = x["Тип жилищного фонда"]
        
        res_data["lat"] = x["lat"]
        res_data["lng"] = x["lng"]
        
        res_data["appeals_count"] = x["appeals_count"]
        res_data["works_count"] = x["works_count"]
        
        res_data["Серия проекта"] = x["Серия проекта"].astype(str)
        
        preds = []
        ps = []
        x_for_pred = x[base_features + final_features + time_features].fillna(-9999)
        x_for_pred = x_for_pred.replace(False, 0)
        
        for i in range(len(all_works)):
            if EPS_LOC[i] == 1:
                preds.append([0 for _ in range(len(x))])
                ps.append([0 for _ in range(len(x))])
            else:
                model = CatBoostClassifier()      # parameters not required.
                model.load_model(f'{models_source}/{i}')
                preds.append((model.predict_proba(x_for_pred)[:, 1] > EPS_LOC[i] / 1000).astype(int))
                ps.append(model.predict_proba(x_for_pred)[:, 1])
        target = []
        ps_target = []
        for i in range(len(res_data)):
            target.append([])
            ps_target.append([])
            for j in range(len(all_works)):
                if preds[j][i]:
                    target[i].append(all_works[j])
                    ps_target[i].append(ps[j][i])
        res_data["target"] = target
        res_data["p"] = ps_target
        res_data = res_data[res_data["target"].apply(lambda x: len(x)) > 0]
        bounds_dict = {"appeals_count": {"type": "float", "bounds": (res_data["appeals_count"].min(),
                                                                     res_data["appeals_count"].max())},
                       "works_count": {"type": "float", "bounds": (res_data["works_count"].min(),
                                                                   res_data["works_count"].max())},
                       "Серия проекта": {"type": "string", "bounds": list(set(res_data["Серия проекта"]))}}
        res_data.to_csv(resultsname, index=False)
        res_data.to_csv(add_name, index=False)
        with open(boundsname, 'w') as f:
            json.dump(bounds_dict, f, ensure_ascii=False, default=str)

In [8]:
geo = pd.read_csv(geo_source)
houses_df, appeals_df, works_df = get_datasets(houses_postprocess_source, appeals_postprocess_source, works_postprocess_source)
data = preprocess(houses_df, appeals_df, works_df, geo)

C:\Users\peter\AppData\Local\Temp\ipykernel_32688\499639984.py:12: DtypeWarning: Columns (22,23,30) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(houses_source)


In [9]:
imply_dict_data, train_data, test_data = split_data(data)

In [40]:
houses_test = houses_df[houses_df["UNOM"].isin(test_data["unom"])]
houses_test.to_csv(houses_test_source, index=False)

In [77]:
imply_appeals_work_without_time_weight_dict = get_imply_appeals_work(imply_dict_data)
imply_appeals_work_with_time_weight_dict = get_imply_appeals_work(imply_dict_data, weight_func=lambda x, y: 1 / (abs(x - y) + 1))

In [78]:
with open(imply_appeals_work_without_time_weight_source, 'wb') as outfile:
    pickle.dump(imply_appeals_work_without_time_weight_dict, outfile)
    
with open(imply_appeals_work_with_time_weight_source, 'wb') as outfile:
    pickle.dump(imply_appeals_work_with_time_weight_dict, outfile)

In [87]:
with open(imply_appeals_work_without_time_weight_source, 'rb') as infile:
    imply_appeals_work_without_time_weight_dict = pickle.load(infile)
    
with open(imply_appeals_work_with_time_weight_source, 'rb') as infile:
    imply_appeals_work_with_time_weight_dict = pickle.load(infile)

In [85]:
imply_appeals_work_without_time_weight_data = create_imply_dict_data(imply_appeals_work_without_time_weight_dict)
imply_appeals_work_with_time_weight_data = create_imply_dict_data(imply_appeals_work_with_time_weight_dict)



In [80]:
imply_appeals_work_without_time_weight_data.sum(axis=1)

Отсутствие связи                                           1.0
Запах газа в подъезде и на улице                           1.0
Запах газа от ШБДГ                                         1.0
Загрязнение/замусоренность подъезда                        1.0
Повреждение козырька подъезда                              1.0
                                                          ... 
Запах газа на улице; Запах газа в доме                     1.0
Гул (шум) на объекте АО МОСГАЗ                             1.0
Запах газа от газового оборудования; Запах газа в доме     0.0
Запах газа на улице и в подъезде                           0.0
Запах газа в кухне; Запах газа от газового оборудования    1.0
Length: 265, dtype: float64

In [81]:
imply_appeals_work_with_time_weight_data.sum(axis=1)

Отсутствие связи                                           1.0
Запах газа в подъезде и на улице                           1.0
Запах газа от ШБДГ                                         1.0
Загрязнение/замусоренность подъезда                        1.0
Повреждение козырька подъезда                              1.0
                                                          ... 
Запах газа на улице; Запах газа в доме                     1.0
Гул (шум) на объекте АО МОСГАЗ                             1.0
Запах газа от газового оборудования; Запах газа в доме     0.0
Запах газа на улице и в подъезде                           0.0
Запах газа в кухне; Запах газа от газового оборудования    1.0
Length: 265, dtype: float64

In [18]:
imply_appeals_work_without_time_weight_data

,ремонт внутридомовых инженерных систем горячего водоснабжения (стояки),замена лифтового оборудования,ремонт пожарного водопровода,ремонт крыши,ремонт мусоропровода,ремонт внутридомовых инженерных систем газоснабжения,ремонт внутридомовых инженерных систем холодного водоснабжения (разводящие магистрали),"ремонт подвальных помещений, относящихся к общему имуществу в многоквартирном доме","замена оконных блоков, расположенных в помещениях общего пользования",ремонт внутридомовых инженерных систем водоотведения (канализации) (выпуски и сборные трубопроводы),ремонт внутридомовых инженерных систем электроснабжения,ремонт фасадов,ремонт внутридомовых инженерных систем теплоснабжения (стояки),ремонт внутридомовых инженерных систем холодного водоснабжения (стояки),ремонт внутридомовых инженерных систем водоотведения (канализации) (стояки),ремонт внутреннего водостока,ремонт внутридомовых инженерных систем горячего водоснабжения (разводящие магистрали),ремонт внутридомовых инженерных систем теплоснабжения (разводящие магистрали),"ремонт подъездов, направленный на восстановление их надлежащего состояния и проводимый при выполнении иных работ по капитальному ремонту общего имущества в многоквартирном доме",without work
Отсутствие связи,0.0,0.015789,0.0,0.004386,0.0,0.0,0.0,0.011404,0.004386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001754,0.009649,0.952632
Запах газа в подъезде и на улице,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Запах газа от ШБДГ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Загрязнение/замусоренность подъезда,0.00117,0.007079,0.0,0.005197,0.000132,0.000217,0.001265,0.00699,0.000763,0.001166,0.001549,0.003886,0.000254,0.00117,0.0,0.001651,0.001659,0.003607,0.001265,0.960981
Повреждение козырька подъезда,0.0,0.007273,0.0,0.005455,0.0,0.0,0.0,0.005455,0.0,0.0,0.0,0.005455,0.0,0.0,0.0,0.003636,0.0,0.001818,0.0,0.970909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Запах газа на улице; Запах газа в доме,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Гул (шум) на объекте АО МОСГАЗ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Запах газа от газового оборудования; Запах газа в доме,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Запах газа на улице и в подъезде,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [82]:
imply_appeals_work_without_time_weight_data

,ремонт внутридомовых инженерных систем горячего водоснабжения (стояки),замена лифтового оборудования,ремонт пожарного водопровода,ремонт крыши,ремонт мусоропровода,ремонт внутридомовых инженерных систем газоснабжения,ремонт внутридомовых инженерных систем холодного водоснабжения (разводящие магистрали),"ремонт подвальных помещений, относящихся к общему имуществу в многоквартирном доме","замена оконных блоков, расположенных в помещениях общего пользования",ремонт внутридомовых инженерных систем водоотведения (канализации) (выпуски и сборные трубопроводы),ремонт внутридомовых инженерных систем электроснабжения,ремонт фасадов,ремонт внутридомовых инженерных систем теплоснабжения (стояки),ремонт внутридомовых инженерных систем холодного водоснабжения (стояки),ремонт внутридомовых инженерных систем водоотведения (канализации) (стояки),ремонт внутреннего водостока,ремонт внутридомовых инженерных систем горячего водоснабжения (разводящие магистрали),ремонт внутридомовых инженерных систем теплоснабжения (разводящие магистрали),"ремонт подъездов, направленный на восстановление их надлежащего состояния и проводимый при выполнении иных работ по капитальному ремонту общего имущества в многоквартирном доме",without work
Отсутствие связи,0.0,0.015789,0.0,0.004386,0.0,0.0,0.0,0.011404,0.004386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001754,0.009649,0.952632
Запах газа в подъезде и на улице,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Запах газа от ШБДГ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Загрязнение/замусоренность подъезда,0.00117,0.007079,0.0,0.005197,0.000132,0.000217,0.001265,0.00699,0.000763,0.001166,0.001549,0.003886,0.000254,0.00117,0.0,0.001651,0.001659,0.003607,0.001265,0.960981
Повреждение козырька подъезда,0.0,0.007273,0.0,0.005455,0.0,0.0,0.0,0.005455,0.0,0.0,0.0,0.005455,0.0,0.0,0.0,0.003636,0.0,0.001818,0.0,0.970909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Запах газа на улице; Запах газа в доме,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Гул (шум) на объекте АО МОСГАЗ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
Запах газа от газового оборудования; Запах газа в доме,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Запах газа на улице и в подъезде,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [212]:
train, y_train, final_features = generate_x_y(train_data)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [21:23<00:00, 12.83s/it]


In [213]:
train.to_csv(f'{data_postprocessed_source}/train.csv', index=False)
y_train.to_csv(f'{data_postprocessed_source}/y_train.csv', index=False)

In [214]:
y_train.mean()

ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)                                                                                                              0.000337
замена лифтового оборудования                                                                                                                                                       0.008485
ремонт пожарного водопровода                                                                                                                                                        0.000000
ремонт крыши                                                                                                                                                                        0.006653
ремонт мусоропровода                                                                                                                                                                0.000377
ремонт внутридомовых инженерных систем газоснабжения   

In [218]:
final_features

['appeals_count',
 'works_count',
 'ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)_last_time_lag',
 'ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)_without_time_weight_sum',
 'ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)_without_time_weight_after_last_work_sum',
 'ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)_without_time_weight_after_last_work_max',
 'ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)_with_time_weight_sum',
 'ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)_with_time_weight_after_last_work_sum',
 'ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)_with_time_weight_after_last_work_max',
 'замена лифтового оборудования_last_time_lag',
 'замена лифтового оборудования_without_time_weight_sum',
 'замена лифтового оборудования_without_time_weight_after_last_work_sum',
 'замена лифтового оборудования_without_time_wei

In [253]:
results_dict = dict()
for work in tqdm(all_works):
    X = train[base_features + final_features + time_features].fillna(-9999)
    y = y_train[work].astype(int)
    groups = train['unom']
    model = CatBoostClassifier(silent=True)
    predictions, test_labels = cv_predictions(model, X, y, groups, n_splits=3)
    if not isinstance(predictions, type(np.array([]))):
        continue
    results_dict[work] = []
    thresholds = np.logspace(-10, 0, 100)
    for threshold in thresholds:
        y_pred = (predictions[:, 1] > threshold).astype(int)
        f1_score_res = f1_score(test_labels, y_pred)
        precision_score_res = precision_score(test_labels, y_pred)
        recall_score_res = recall_score(test_labels, y_pred)
        roc_auc_score_res = roc_auc_score(test_labels, predictions[:, 1])
        results_dict[work].append((threshold, f1_score_res, precision_score_res, recall_score_res,
                                   roc_auc_score_res))


100%|███████████████████████████████████████████████████████████████████████████████| 19/19 [1:35:10<00:00, 300.55s/it]


In [252]:
isinstance(predictions, type(np.array([])))

True

In [254]:
with open("results.pkl", 'wb') as outfile:
    pickle.dump(results_dict, outfile)

In [269]:
EPS_LOC = []
for work in all_works:
    if work not in results_dict:
        EPS_LOC.append(1)
    else:
        print(work)
        max_precision = max(results_dict[work], key=lambda x: x[2])
        print(max_precision)
        if max_precision[4] < 0.8 or max_precision[2] < 0.3:
            EPS_LOC.append(1)
        else:
            EPS_LOC.append(max_precision[0])

ремонт внутридомовых инженерных систем горячего водоснабжения (стояки)
(0.7924828983539186, 0.10526315789473684, 0.5, 0.058823529411764705, 0.9934861055815719)
замена лифтового оборудования
(0.15556761439304723, 0.3351293103448275, 0.5427574171029669, 0.24240062353858144, 0.8970600313103362)
ремонт крыши
(0.7924828983539186, 0.6775320139697322, 0.8174157303370787, 0.5785288270377733, 0.9531986212344957)
ремонт внутридомовых инженерных систем газоснабжения
(0.0002915053062825176, 0.022408963585434174, 0.013201320132013201, 0.07407407407407407, 0.7426455772347679)
ремонт внутридомовых инженерных систем холодного водоснабжения (разводящие магистрали)
(0.0007390722033525775, 0.015625, 0.011904761904761904, 0.022727272727272728, 0.8550966612667305)
ремонт подвальных помещений, относящихся к общему имуществу в многоквартирном доме
(0.7924828983539186, 0.7927356586912655, 0.8724619289340102, 0.7263602746962493, 0.9682113367632383)
замена оконных блоков, расположенных в помещениях общего польз

In [261]:
for i in tqdm(range(len(all_works))):
    if all_works[i] not in results_dict:
        continue
    X = train[base_features + final_features + time_features].fillna(-9999)
    y = y_train[all_works[i]].astype(int)
    model = CatBoostClassifier(silent=True, allow_const_label=True)
    model.fit(X, y, cat_features=base_cat_features)
    model.save_model(f'{models}/{i}')
    
    

100%|█████████████████████████████████████████████████████████████████████████████████| 19/19 [40:03<00:00, 126.50s/it]
